In [48]:
import pandas as pd

# Define los paths de los archivos Excel y los nombres de las jornadas
jugadores_df_paths = [
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J1_jugadores.xlsx',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J2_jugadores.xlsx',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J3_jugadores.xlsx',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J4_jugadores.xlsx',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J5_jugadores.xlsx'
]

nombres_jornadas = {
    "J1": "Jornada 1 - Local vs Universidad Cesar Vallejo",
    "J2": "Jornada 2 - Visita vs Alianza Atlético de Sullana",
    "J3": "Jornada 3 - Local vs Universitario de Deportes",
    "J4": "Jornada 4 - Visita vs Unión Comercio",
    "J5": "Jornada 5 - Local vs Comerciantes Unidos",
}

# Lista para almacenar los DataFrames de cada jornada
dfs = []

# Leer cada archivo y agregarlo a la lista de DataFrames
for path, jornada in zip(jugadores_df_paths, nombres_jornadas):
    df_temp = pd.read_excel(path)
    df_temp['Jornada'] = nombres_jornadas[jornada]  # Añade la columna 'Jornada'
    dfs.append(df_temp)

# Combinar todos los DataFrames en uno solo
df_combinado = pd.concat(dfs, ignore_index=True)

# Exportar a Excel
df_combinado.to_excel(r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\Resumen_AL_Jugadores.xlsx', index=False)


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Carga de datos
df = pd.read_excel(r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX finales\Resumen_AL_Jugadores.xlsx')

# Creación de widget para selección de jugador
jugador_selector = widgets.Dropdown(
    options=sorted(df['Nombre'].unique()),  # Orden alfabético
    description='Jugador:',
)

# Widget de salida
output = widgets.Output()

def generar_informe(jugador_seleccionado):
    with output:
        clear_output(wait=True)
        df_jugador = df[df['Nombre'] == jugador_seleccionado].copy()

        # Asegurar que los nombres de las jornadas coincidan con los esperados
        df_jugador['Jornada'] = df_jugador['Jornada'].map({
            'Jornada 1 - Local vs Universidad Cesar Vallejo': 'J1',
            'Jornada 2 - Visita vs Alianza Atlético de Sullana': 'J2',
            'Jornada 3 - Local vs Universitario de Deportes': 'J3',
            'Jornada 4 - Visita vs Unión Comercio': 'J4',
            'Jornada 5 - Local vs Comerciantes Unidos': 'J5'
        })
        promedio_pases_acertados = df['Pases Acertados'].mean()
        promedio_balones_largos_acertados = df['Balones Largos Acertados'].mean()
        promedio_centros_acertados = df['Centros Acertados'].mean()
        color_fondo = 'black'
        color_linea_promedio = 'white'
        estadisticas_y_promedios = [
            ('Pases Acertados', promedio_pases_acertados, 'Total de Pases'),
            ('Balones Largos Acertados', promedio_balones_largos_acertados, 'Total de Balones Largos'),
            ('Centros Acertados', promedio_centros_acertados, 'Total de Centros')
        ]
        # Establecer ancho de las barras
        bar_width = 0.3

        # Creación de gráficos
        fig, axs = plt.subplots(3, 1, figsize=(10, 18), facecolor=color_fondo)
        for i, (estadistica, promedio, total_estadistica) in enumerate(estadisticas_y_promedios):
            ax = axs[i]
            # Posiciones de las barras para cada jornada
            r1 = np.arange(len(df_jugador['Jornada']))
            r2 = [x + bar_width for x in r1]

            # Dibujar barras para "Acertados" y "Total"
            ax.bar(r1, df_jugador[estadistica], color='lightblue', width=bar_width, edgecolor='steelblue', label=estadistica)
            ax.bar(r2, df_jugador[total_estadistica], color='grey', width=bar_width, edgecolor='navy', label=total_estadistica)

            # Dibujar línea de promedio
            ax.axhline(y=promedio, color=color_linea_promedio, linestyle='--', label=f'Promedio del equipo: {promedio:.2f}')

            ax.set_facecolor(color_fondo)
            ax.set_title(f'{estadistica} por Jornada para {jugador_seleccionado}', color='white')
            ax.set_xlabel('Jornada', color='white')
            ax.set_xticks([r + bar_width / 2 for r in range(len(r1))], df_jugador['Jornada'])
            ax.set_ylabel(estadistica, color='white')
            ax.tick_params(colors='white')
            ax.grid(color='gray', linestyle='--', linewidth=0.5)
            ax.legend(facecolor=color_fondo, edgecolor=color_fondo)
            for text in ax.legend().get_texts():
                text.set_color('black')

        plt.tight_layout()
        plt.show()


def on_jugador_seleccionado(change):
    generar_informe(change.new)

jugador_selector.observe(on_jugador_seleccionado, names='value')

display(jugador_selector)
display(output)

Dropdown(description='Jugador:', options=('Adrián Arregui', 'Aldair Fuentes', 'Axel Moyano', 'Catriel Cabellos…

Output()